A* algorithm performing 8-puzzle problem


In [5]:
import numpy
import math
import random
import copy


class Node:
    def __init__(self, state):
        self.target = [1,2,3,4,5,6,7,8,0]
        self.state = copy.deepcopy(state)
        self.dim = 3
        self.nextstates = []
        self.weightedstates = None
        self.flatstate = numpy.ravel(self.state)
        self.zeroindex = numpy.where(self.flatstate == 0)[0][0]
        self.zerorow = math.floor(self.zeroindex / self.dim)
        self.zerocol = self.zeroindex % self.dim
        self.genstates()
        self.getheuristicCost()

    def swap(self, i):
        r = self.zerorow
        c = self.zerocol
        if i == 'u':
            node = copy.deepcopy(self.state)
            node[r][c], node[r - 1][c] = node[r - 1][c], node[r][c]
            self.nextstates.append(node)

        if i == 'd':
            node = copy.deepcopy(self.state)
            node[r][c], node[r + 1][c] = node[r + 1][c], node[r][c]
            self.nextstates.append(node)

        if i == 'r':
            node = copy.deepcopy(self.state)
            node[r][c], node[r][c + 1] = node[r][c + 1], node[r][c]
            self.nextstates.append(node)

        if i == 'l':
            node = copy.deepcopy(self.state)
            node[r][c], node[r][c - 1] = node[r][c - 1], node[r][c]
            self.nextstates.append(node)

    def genstates(self):
        directions = ['u', 'd', 'r', 'l']
        dr = [-1, 1, 0, 0]
        dc = [0, 0, 1, -1]
        for i in range(4):
            rr = self.zerorow + dr[i]
            cc = self.zerocol + dc[i]
            if rr < 0 or cc < 0 or rr >= self.dim or cc >= self.dim:
                continue
            self.swap(directions[i])

    def getheuristicCost(self):
        totalcost = []
        for j in self.nextstates:
            flatstate = numpy.ravel(j)
            statecost = 0
            for i, v in enumerate(self.target[:-1]):
                targetrow = math.floor(i / self.dim)
                targetcol = i % self.dim
                numindex = numpy.where(flatstate == v)[0][0]
                numrow = math.floor(numindex / self.dim)
                numcol = numindex % self.dim
                statecost += math.fabs(targetrow - numrow)
                statecost += math.fabs(targetcol - numcol)
            statecost += 1
            totalcost.append([j, statecost])
        self.weightedstates = copy.deepcopy(totalcost)


class Memory:
    def __init__(self, initialstate):
        self.target = [[1,2,3],[4,5,6],[7,8,0]]
        self.nodesvisited = []
        self.prioqueue = [initialstate]

    def setprioqueue(self, states):
        for i in states:
            if i[0] in self.nodesvisited or i in self.prioqueue:
                continue
            self.prioqueue.append(i)

    def getnextstate(self):
        if len(self.prioqueue) == 1:
            return self.prioqueue[0]
        less = self.prioqueue[0]
        for i in self.prioqueue:
            if i[1] < less[1]:
                less = i
        return less[0]

    def popprioqueue(self, item):
        if len(self.nodesvisited) == 1:
            self.prioqueue.pop(-1)
        for i in self.prioqueue:
            if i[0] == item:
                self.prioqueue.pop(self.prioqueue.index(i))

def Astar(memory):
    count = 0
    while memory.prioqueue:
        count += 1
        boardconfiguration = Node(memory.getnextstate())
        memory.nodesvisited.append(boardconfiguration.state)
        memory.popprioqueue(memory.nodesvisited[-1])
        memory.setprioqueue(boardconfiguration.weightedstates)
        if memory.target == boardconfiguration.state:
            end = boardconfiguration.state
            break
    return f'target: {end}, steps:{count}'

firststate = [[1,2,3],[4,5,0],[6,7,8]]
memory = Memory(initialstate=firststate)
solution = Astar(memory)
print(solution)


target: [[1, 2, 3], [4, 5, 6], [7, 8, 0]], steps:99


Results: A algorithm performs 8-puzzle problem(and many others) very well, even some random starts taking longer, own proved completeness show contrastant results agains breadth-first-search or depth-first-search. Thus algorithm time complexity is mostly defined by his heuristic, whose implementation in this problem were calculating each field distance from target's field position, from each state.